In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tidy3d as td
import tidy3d.web as web


# --- 1. Material Definitions ---
# Using fixed indices for 1550nm for simplicity

wavelength =  1.55                              # Central wavelength
wv_points = 50                   # number of wavelength wv_points
pos_points = 15
width = 0.35
n_si = 3.475
n_sio2 = 1.458
gap_points = 5

sweep_wavelength = np.linspace(1.5,1.6,wv_points)  # Sweep on wavelengths
sweep_freq = td.C_0 / sweep_wavelength          # Sweep on Frequencies
bandwidth = sweep_wavelength[-1]-sweep_wavelength[0]
sweep_monitor_pos = np.linspace(-6,6,pos_points)
sweep_gap = np.linspace(0.15,0.25,gap_points)

mat_si = td.material_library["cSi"]["Li1993_293K"] # Material trace permitivity model for crystaline Silicon
mat_sio2 = td.material_library["SiO2"]["Horiba"]   # Material trace permitivity model for crystaline Silica


web.configure("U7IS5Q1AW3qYVxPAAQDE7vgxaTz6Kt6aI9AioPZUajVRQQ2o")

In [ ]:
def build_MMI_simulation(
    gap=np.array([0.15]),
    width=0.350,
    thickness=np.array([0.22,0.3]),
    wavelength = np.array([1.55]),
):

    # Frequency
    freq = td.C_0 / wavelength

    # Materials
    core = mat_si
    clad = mat_sio2

    # --- We define the simulation data array and simulation objects for the two different sweeps----

    sim_data_arr = [[],[]] # Simulation data for 220nm and 300nm
    sim_arr = [[],[]]      # Simulation objects for 220nm and 300nm
    estimate = 0

    #--- Define Sources ---
    freq0 = td.C_0 / wavelength.mean()
    fwidth = (td.C_0 / wavelength.min()) - (td.C_0 / wavelength.max())


    for thick_idx,thick in enumerate(thickness):
        for gap_idx,gap_values in enumerate(gap):

            # --- Define two silicon rails ---
            rail_offset = width/2 + gap_values/2


            rail_right = td.Structure(
                geometry=td.Box(
                    center=(0, rail_offset, 0),
                    size =(td.inf, width, thick),
                ),
                medium=core,
            )

            rail_left = td.Structure(
                geometry=td.Box(
                    center=(0, -rail_offset, 0),
                    size=(td.inf, width, thick),
                ),
                medium=core,
            ) # Bien definido hasta aca


            source1 = td.ModeSource(
            name = "Mode_source",
            center = [-7.5, rail_offset, 0],
            size = [0, width + gap_values / 2, thick + 0.5],
            source_time = td.GaussianPulse(freq0 = freq0, fwidth =fwidth,),
            direction = '+',
            mode_index=0,
            mode_spec= td.ModeSpec(num_modes=5,target_neff=n_si),
            )


            # --- Define monitors ---

            Through = td.FieldMonitor(
            name="Through",
            center = [2, rail_offset,0],
            size = [0, width + gap_values / 2, thick + 0.5],
            freqs = freq,
            )

            Cross = td.FieldMonitor(
            name="Cross",
            center = [2, -rail_offset, 0],
            size = [0, width + gap_values / 2, thick + 0.5],
            freqs = freq,
            )


            Both = td.FieldMonitor(
            name="Both",
            center = [2,0,0],
            size = [0, 2, 2],
            freqs = freq,
            )

            Longitudinal = td.FieldMonitor(
                name="Longitudinal",
                size = [td.inf, 7.75, 0],
                freqs = freq,
            )# Bien hasta aca

            # --- Simulation domain ---
            sim_arr[thick_idx].append(td.Simulation(
                size=(15.5, 15.5, 15.5),  # x ignored for mode solver
                monitors=[Cross,Through,Longitudinal,Both],
                sources=[source1],
                structures=[rail_right, rail_left],
                medium=clad,
                symmetry=(0, 0, 1),  # symmetry in z
                run_time=1e-12,
                grid_spec=td.GridSpec.auto(wavelength = wavelength.min(),min_steps_per_wvl = 11),
            ))



            Job = web.Job(simulation=sim_arr[thick_idx][gap_idx], task_name="my_sim")

            estimate+= Job.estimate_cost()

            sim_data_arr[thick_idx].append(web.run(sim_arr[thick_idx][gap_idx], task_name='DC_FDTD'))
    print(f"Estimated Maximum Cost: {estimate}")

    return sim_data_arr, sim_arr


sim_data, sim = build_slot_simulation(gap = sweep_gap,wavelength=sweep_wavelength)